# Karınca Kolonisi Algoritması ile Yol Optimizasyonu

**Adınız:** [Adınızı buraya yazın]

**Soyadınız:** [Soyadınızı buraya yazın]

**Okul Numaranız:** [Okul numaranızı buraya yazın]

**GitHub Repo Bağlantısı:** https://github.com/kullanici_adi/aco_yol_optimizasyonu

---

## Senaryo 4: Antalya Muratpaşa Kargo Firması

Antalya'nın Muratpaşa ilçesindeki bir kargo firması 20 farklı mağazaya günde 1 kez uğramak zorundadır. En kısa rotayı seçiniz.


## 1. Gerekli Kütüphanelerin Yüklenmesi


In [ ]:
# Gerekli kütüphaneleri yükle
!pip install streamlit googlemaps pandas numpy matplotlib folium streamlit-folium python-dotenv PyDrive2 -q


## 2. Google Drive Bağlantısı


In [ ]:
# Google Drive'ı bağla
from google.colab import drive
drive.mount('/content/drive')


## 3. Proje Dosyalarının Yüklenmesi


In [ ]:
# Proje dosyalarını Google Drive'dan kopyala veya doğrudan yükle
# Alternatif: GitHub'dan clone et
# !git clone https://github.com/kullanici_adi/aco_yol_optimizasyonu.git
# %cd aco_yol_optimizasyonu

# Veya dosyaları manuel olarak yükleyin
import os
os.chdir('/content')


## 4. Google Maps API Key Ayarları


In [ ]:
# API key'inizi buraya girin (güvenlik için Colab'ın secrets özelliğini kullanabilirsiniz)
import os
os.environ['GOOGLE_MAPS_API_KEY'] = 'YOUR_API_KEY_HERE'  # API key'inizi buraya yazın


## 5. Streamlit Uygulamasını Çalıştırma

**Not:** Colab'da Streamlit çalıştırmak için iki yöntem var:
1. **Yöntem A (Önerilen):** Colab'ın yerleşik port forwarding özelliğini kullan
2. **Yöntem B:** Alternatif olarak doğrudan Python ile çalıştır (Cell 11'den itibaren)


In [ ]:
# YÖNTEM A: Streamlit'i Colab'da çalıştırma (Port Forwarding ile)
# Bu yöntem Colab'ın yerleşik port forwarding özelliğini kullanır

import subprocess
import sys
import time
from IPython.display import HTML, clear_output

print("🚀 Streamlit uygulaması başlatılıyor...")

# Streamlit'i arka planda başlat
def run_streamlit():
    cmd = [
        sys.executable, "-m", "streamlit", "run", "main.py",
        "--server.port", "8501",
        "--server.headless", "true",
        "--server.enableCORS", "false",
        "--server.enableXsrfProtection", "false",
        "--browser.gatherUsageStats", "false"
    ]
    process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    return process

# Streamlit'i başlat
streamlit_process = run_streamlit()

# Biraz bekle (Streamlit'in başlaması için)
time.sleep(5)

# Colab'ın port forwarding özelliğini kullan
print("✅ Streamlit başlatıldı!")
print("\n" + "="*60)
print("📌 UYGULAMAYA ERİŞİM:")
print("="*60)
print("\n1. Colab'ın sağ üst köşesindeki '🔗' (port forwarding) ikonuna tıklayın")
print("2. Açılan pencerede '8501' portunu seçin")
print("3. Veya aşağıdaki linke tıklayın (bazı tarayıcılarda çalışmayabilir)")
print("\n" + "="*60)

# Alternatif: get_ipython ile port forwarding
try:
    from google.colab import output
    output.serve_kernel_port_as_iframe(8501, height=800)
    print("\n✅ Port forwarding aktif! Yukarıdaki iframe'de uygulama görünecektir.")
except:
    print("\n⚠️ Otomatik port forwarding başarısız. Manuel olarak yukarıdaki adımları izleyin.")

print("\n" + "="*60)
print("⚠️ UYARI: Uygulamayı durdurmak için kernel'i yeniden başlatın (Runtime > Restart runtime)")
print("="*60)


## 6. Alternatif: Doğrudan Python ile Çalıştırma

Eğer Streamlit kullanmak istemiyorsanız, algoritmayı doğrudan çalıştırabilirsiniz:


In [ ]:
# Örnek kullanım
from data.coordinates import load_data_from_drive, create_sample_data, get_coordinates_from_dataframe
from core.matrix_utils import calculate_distance_matrix_google_maps, calculate_distance_matrix_haversine
from core.ant_algorithm import AntColonyOptimizer
from visual.plotting import create_route_map, plot_convergence
import config

# Veri yükle
GOOGLE_DRIVE_FOLDER_ID = "1X6f9c4m4p-50gFVcmxLHYfnhHe5tJFbT"

try:
    df = load_data_from_drive(GOOGLE_DRIVE_FOLDER_ID)
except:
    df = create_sample_data()

names, latitudes, longitudes = get_coordinates_from_dataframe(df)
coordinates = list(zip(latitudes, longitudes))

print(f"{len(coordinates)} nokta yüklendi")
print(df.head())


In [ ]:
# Mesafe matrisini hesapla
try:
    from core.matrix_utils import initialize_google_maps_client
    client = initialize_google_maps_client()
    distance_matrix, duration_matrix = calculate_distance_matrix_google_maps(coordinates, client)
    print("Google Maps API ile mesafe matrisi oluşturuldu")
except Exception as e:
    print(f"Google Maps API hatası: {e}")
    print("Haversine formülü kullanılıyor...")
    distance_matrix = calculate_distance_matrix_haversine(coordinates)

print(f"Mesafe matrisi boyutu: {distance_matrix.shape}")


In [ ]:
# ACO algoritmasını çalıştır
optimizer = AntColonyOptimizer(
    distance_matrix=distance_matrix,
    n_ants=50,
    n_iterations=100,
    alpha=1.0,
    beta=2.0,
    evaporation_rate=0.5
)

best_path, best_distance, iteration_distances = optimizer.solve(start_city=0)

print(f"En kısa rota: {best_distance:.2f} km")
print(f"Rota: {[names[i] for i in best_path]}")


In [ ]:
# Sonuçları görselleştir
import os
os.makedirs('figure', exist_ok=True)

route_map = create_route_map(coordinates, best_path, names, best_distance)
route_map.save('figure/rota.html')
print("Harita kaydedildi: figure/rota.html")

# Yakınsama grafiği
plot_convergence(iteration_distances, 'figure/convergence.png')
print("Yakınsama grafiği kaydedildi: figure/convergence.png")


## 7. Sonuçlar

Proje başarıyla tamamlandı! Sonuçlar `figure/` klasöründe kaydedildi.
